# training-test data 60-40, 70-30,80-20,90-10

In [1]:
from glob import glob
import os
import mne
import numpy as np
import pandas as pd
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from sklearn.linear_model import LogisticRegression

In [2]:
no_of_patients=109

In [3]:
train=[]
test=[]
for i in range(1,no_of_patients+1):
    files=glob('C:/aditya/Research_Intership/eeg_files/files/S'+str(str(i).zfill(3))+'/*.edf')
    files=files[2:3]
    train+=files
train

['C:/aditya/Research_Intership/eeg_files/files/S001\\S001R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S002\\S002R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S003\\S003R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S004\\S004R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S005\\S005R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S006\\S006R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S007\\S007R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S008\\S008R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S009\\S009R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S010\\S010R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S011\\S011R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S012\\S012R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S013\\S013R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S014\\S014R03.edf',
 'C:/aditya/Research_Intership/eeg_files/files/S015\\S015R03.e

In [4]:
def read_data(i,train_split):
    raw = mne.io.read_raw_edf(i, preload=True)
    eeg_data = raw.get_data()
    eeg_channels = [f'Channel_{i}' for i in range(eeg_data.shape[0])]
    eeg_df = pd.DataFrame(data=eeg_data.T, columns=eeg_channels)
#     display(eeg_df)
    eeg_df = eeg_df.iloc[:15000]
    eeg_df.sample()
    
    idx1= int(train_split*(len(eeg_df)))
    idx2= int(train_split*(len(eeg_df)))+1
        
    eeg_df1=eeg_df.iloc[:idx1]
    eeg_df2=eeg_df.iloc[idx2:]
    
    return eeg_df1,eeg_df2, len(eeg_df1),len(eeg_df2)

In [5]:
def scale_dataset(dataframe):
    x=dataframe[dataframe.columns[:-1]].values
    y=dataframe[dataframe.columns[-1]].values
    scaler =StandardScaler()
    x=scaler.fit_transform(x)
    return x,y

## 60-40

In [6]:
%%capture
xtemp1=[]
xtemp2=[]
ytemp1=[]
ytemp2=[]
for i in range(no_of_patients):
    xtr,xte,ytr,yte=read_data(train[i],0.6) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    ytemp1.append(ytr)
    ytemp2.append(yte)

xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)

ytrain=[]
for i in range(1,len(ytemp1)+1):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(1,len(ytemp2)+1):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        

xtrain['id']=ytrain
xtest['id']=ytest
# display(xtrain)
x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)

lr_model=LogisticRegression()
lr_model.fit(x,y)


In [7]:
y_pred=lr_model.predict(xt)
print(classification_report(yt,y_pred))

              precision    recall  f1-score   support

           1       0.12      0.06      0.08      5999
           2       0.13      0.09      0.11      5999
           3       0.19      0.24      0.21      5999
           4       0.78      0.76      0.77      5999
           5       0.09      0.08      0.09      5999
           6       0.20      0.37      0.26      5999
           7       0.44      0.70      0.54      5999
           8       0.07      0.03      0.04      5999
           9       0.10      0.11      0.10      5999
          10       0.07      0.07      0.07      5999
          11       0.07      0.07      0.07      5999
          12       0.27      0.29      0.28      5999
          13       0.10      0.12      0.11      5999
          14       0.11      0.11      0.11      5999
          15       0.06      0.03      0.04      5999
          16       0.36      0.29      0.32      5999
          17       0.00      0.00      0.00      5999
          18       0.49    

## 70-30

In [8]:
%%capture
xtemp1=[]
xtemp2=[]
ytemp1=[]
ytemp2=[]
for i in range(no_of_patients):
    xtr,xte,ytr,yte=read_data(train[i],0.7) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    ytemp1.append(ytr)
    ytemp2.append(yte)

xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)

ytrain=[]
for i in range(1,len(ytemp1)+1):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(1,len(ytemp2)+1):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        

xtrain['id']=ytrain
xtest['id']=ytest

x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)

lr_model=LogisticRegression()
lr_model.fit(x,y)


In [9]:
y_pred=lr_model.predict(xt)
print(classification_report(yt,y_pred))

              precision    recall  f1-score   support

           1       0.13      0.05      0.07      4499
           2       0.17      0.10      0.13      4499
           3       0.15      0.20      0.17      4499
           4       0.78      0.76      0.77      4499
           5       0.09      0.09      0.09      4499
           6       0.21      0.41      0.28      4499
           7       0.42      0.69      0.53      4499
           8       0.04      0.02      0.03      4499
           9       0.13      0.16      0.14      4499
          10       0.05      0.06      0.05      4499
          11       0.04      0.04      0.04      4499
          12       0.29      0.31      0.30      4499
          13       0.10      0.12      0.11      4499
          14       0.14      0.15      0.14      4499
          15       0.06      0.03      0.04      4499
          16       0.37      0.27      0.31      4499
          17       0.00      0.00      0.00      4499
          18       0.48    

## 80-20

In [10]:
%%capture
xtemp1=[]
xtemp2=[]
ytemp1=[]
ytemp2=[]
for i in range(no_of_patients):
    xtr,xte,ytr,yte=read_data(train[i],0.8) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    ytemp1.append(ytr)
    ytemp2.append(yte)

xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)

ytrain=[]
for i in range(1,len(ytemp1)+1):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(1,len(ytemp2)+1):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        

xtrain['id']=ytrain
xtest['id']=ytest

x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)

lr_model=LogisticRegression()
lr_model.fit(x,y)


In [11]:
y_pred=lr_model.predict(xt)
print(classification_report(yt,y_pred))

              precision    recall  f1-score   support

           1       0.14      0.05      0.07      2999
           2       0.18      0.11      0.13      2999
           3       0.19      0.28      0.22      2999
           4       0.74      0.84      0.79      2999
           5       0.06      0.07      0.07      2999
           6       0.23      0.43      0.30      2999
           7       0.42      0.71      0.53      2999
           8       0.12      0.05      0.07      2999
           9       0.16      0.21      0.18      2999
          10       0.06      0.05      0.06      2999
          11       0.13      0.09      0.10      2999
          12       0.29      0.31      0.30      2999
          13       0.09      0.10      0.10      2999
          14       0.19      0.19      0.19      2999
          15       0.07      0.03      0.04      2999
          16       0.34      0.23      0.27      2999
          17       0.00      0.00      0.00      2999
          18       0.53    

## 90-10

In [12]:
%%capture
xtemp1=[]
xtemp2=[]
ytemp1=[]
ytemp2=[]
for i in range(no_of_patients):
    xtr,xte,ytr,yte=read_data(train[i],0.9) # xtr=xtrain, xte=xtest, ytr=ytrain, yte=ytest.
    xtemp1.append(xtr)
    xtemp2.append(xte)
    ytemp1.append(ytr)
    ytemp2.append(yte)

xtrain = pd.concat([xtemp1[i] for i in range(0, len(xtemp1))], ignore_index=True)
xtest = pd.concat([xtemp2[i] for i in range(0, len(xtemp2))], ignore_index=True)

ytrain=[]
for i in range(1,len(ytemp1)+1):
    for j in range(ytemp1[i-1]):
        ytrain.append(i)
ytest=[]
for i in range(1,len(ytemp2)+1):
    for j in range(ytemp2[i-1]):
        ytest.append(i)        

xtrain['id']=ytrain
xtest['id']=ytest

x,y=scale_dataset(xtrain)
xt,yt=scale_dataset(xtest)

lr_model=LogisticRegression()
lr_model.fit(x,y)

In [13]:
y_pred=lr_model.predict(xt)
print(classification_report(yt,y_pred))

              precision    recall  f1-score   support

           1       0.06      0.02      0.03      1499
           2       0.17      0.10      0.13      1499
           3       0.13      0.20      0.16      1499
           4       0.70      0.83      0.76      1499
           5       0.07      0.06      0.07      1499
           6       0.28      0.42      0.34      1499
           7       0.38      0.67      0.49      1499
           8       0.27      0.11      0.16      1499
           9       0.24      0.24      0.24      1499
          10       0.02      0.01      0.02      1499
          11       0.05      0.03      0.04      1499
          12       0.25      0.23      0.24      1499
          13       0.08      0.09      0.08      1499
          14       0.19      0.21      0.20      1499
          15       0.07      0.04      0.05      1499
          16       0.09      0.06      0.07      1499
          17       0.00      0.00      0.00      1499
          18       0.49    